In [21]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device 

import copy
import os.path as osp
import cv2

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset 

from mmdet.apis import init_detector, inference_detector

from pycocotools.coco import COCO

In [22]:


@DATASETS.register_module(force=True)
class RecycleDataset(CustomDataset): 
    CLASSES = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing") 

    @staticmethod
    def make_box_coco2voc(bbox): 
        x = bbox[0]
        y = bbox[1]
        x_ = bbox[0] + bbox[2]
        y_ = bbox[1] + bbox[3] 

        return [x, y, x_, y_]

    def load_annotations(self, ann_file): 
        coco = COCO(f'{self.data_root}/annotations/train.json')
        cat2label = {k: i for i, k in enumerate(self.CLASSES)} 
        image_list = mmcv.list_from_file(ann_file)

        image_names = [] 
        for image_name in image_list: 
            image_names.append(image_name) 

        data_infos = [] 
        for image_name in image_names: 
            file_path = f'{self.img_prefix}/{image_name}.jpg' 
            image_id = 0 if image_name.replace('0', '') == '' else int(image_name.replace('0', '')) 

            image = cv2.imread(file_path)
            height, width = image.shape[:2]

            data_info = {
                'filename': image_name + '.jpg', 
                'width': width, 
                'height': height 
            }

            ann_ids = coco.getAnnIds(imgIds=image_id)
            anns = coco.loadAnns(ann_ids)   

            bbox_names = [self.CLASSES[ann['category_id']] for ann in anns]
            bboxes = [list(map(float, ann['bbox'])) for ann in anns] 

            gt_bboxes = [] 
            gt_labels = [] 
            gt_bboxes_ignore = [] 
            gt_labels_ignore = [] 

            for bbox_name, bbox in zip(bbox_names, bboxes): 
                if bbox_name in cat2label: 
                    gt_bboxes.append(self.make_box_coco2voc(bbox)) 
                    gt_labels.append(cat2label[bbox_name]) 
                else: 
                    gt_bboxes_ignore.append(self.make_box_coco2voc(bbox)) 
                    gt_labels_ignore.append(-1) 
            
            
            data_anno = {
                'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4), 
                'labels': np.array(gt_labels, dtype=np.long), 
                'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4), 
                'labels_ignore': np.array(gt_labels_ignore, dtype=np.long) 
            } 

            data_info.update(ann=data_anno) 
            data_infos.append(data_info) 
        
        return data_infos 

In [28]:
from mmdet.apis import set_random_seed

cfg = Config.fromfile('/opt/ml/baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

# dataset에 대한 환경 파라미터 수정. 
cfg.dataset_type = 'RecycleDataset'
cfg.data_root = '/opt/ml/my_proj/data'

# train, val, test dataset에 대한 type, data_root, ann_file, img_prefix 환경 파라미터 수정. 
cfg.data.train.type = 'RecycleDataset'
cfg.data.train.data_root = '/opt/ml/my_proj/data'
cfg.data.train.ann_file = 'train0.txt'
cfg.data.train.img_prefix = 'images'

cfg.data.val.type = 'RecycleDataset'
cfg.data.val.data_root = '/opt/ml/my_proj/data'
cfg.data.val.ann_file = 'val0.txt'
cfg.data.val.img_prefix = 'images'

cfg.data.test.type = 'RecycleDataset'
cfg.data.test.data_root = '/opt/ml/my_proj/data'
cfg.data.test.ann_file = 'val0.txt'
cfg.data.test.img_prefix = 'images'

# cfg.log_config.hooks = [
#     dict(type='TextLoggerHook'),
#     dict(type='MMDetWandbHook',
#          init_kwargs={'project': 'ObjectDetection-MMDetection '},
#          interval=10,
#          log_checkpoint=True,
#          log_checkpoint_metadata=True,
#          num_eval_images=100)]

cfg.data.samples_per_gpu = 4

# class의 갯수 수정. 
cfg.model.roi_head.bbox_head.num_classes = 10

# 학습 weight 파일로 로그를 저장하기 위한 디렉토리 설정. 
cfg.work_dir = './work_dirs/faster_rcnn_r50_base'

# 학습율 변경 환경 파라미터 설정. 
cfg.optimizer.lr = 0.02 / 8

cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# config 수행 시마다 policy값이 없어지는 bug로 인하여 설정. 
cfg.lr_config.policy = 'step'

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# ConfigDict' object has no attribute 'device 오류 발생시 반드시 설정 필요. https://github.com/open-mmlab/mmdetection/issues/7901
cfg.device='cuda'


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_w

In [29]:
from mmdet.datasets import build_dataset 
from mmdet.models import build_detector 
from mmdet.apis import train_detector 

datasets = [build_dataset(cfg.data.train)] 

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2

In [30]:
datasets[0]


RecycleDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3244  | 1 [Paper]     | 5225  | 2 [Paper pack]  | 709   | 3 [Metal]   | 885   | 4 [Glass]    | 917   |
| 5 [Plastic]       | 2281  | 6 [Styrofoam] | 1089  | 7 [Plastic bag] | 4190  | 8 [Battery] | 170   | 9 [Clothing] | 341   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [31]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.init_weights()
model.CLASSES = datasets[0].CLASSES

In [32]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2023-05-07 23:53:53,844 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warn

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


2023-05-07 23:54:07,787 - mmdet - INFO - load checkpoint from local path: /opt/ml/my_proj/faster_rcnn/checkpoints/faster_rcnn_r50_fpn_mstrain_3x_coco_20210524_110822-e10bd31c.pth
2023-05-07 23:54:07,912 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([11, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([11]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([40, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([40]).
2023-05-07 23:54:07,913 - mmdet - INFO - Start running, h

RuntimeError: CUDA out of memory. Tried to allocate 158.00 MiB (GPU 0; 31.75 GiB total capacity; 4.53 GiB already allocated; 58.50 MiB free; 4.71 GiB reserved in total by PyTorch)